In [1]:
from src.hook.countries_hook import CountriesHook
file_format = 'csv'
hook = (CountriesHook()
            .parse_countries()
            .parse_tables()
            .save_tables(file_format, destination='C:\Git\countries_dw\src\dbt\seeds', index=False)
        )
hook 

Countries hook: tables parsed (yes)

In [24]:
# hook.continents.continents.str.len().max()
import numpy as np
import pandas as pd
def max_len_of_columns(df):
    measurer = np.vectorize(len)
    _dict = dict(zip(df.columns, measurer(df.values.astype(str)).max(axis=0)))
    return pd.DataFrame([_dict])
    # return _dict
index = 25
print(hook.tables[index])
df = getattr(hook, hook.tables[index])
max_len_of_columns(df)

# df

IndexError: list index out of range

In [18]:
df.explode('suffixes').query('common_name == "Puerto Rico"')

,common_name,root,suffixes
246,Puerto Rico,+1,787
246,Puerto Rico,+1,939
